# Implementing Multiplex Adamic-Adar Index (Aleta et al., 2016)

## 1. Data Prepraration

In [2]:
from mplexaa import MultiplexGraphDataset
from mplexaa.utils import *
from tabulate import tabulate

datasets = get_datasets()

for dataset, layers in datasets:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    print(tabulate(ds.get_info(), headers="keys", tablefmt="pipe", floatfmt=".2f"))

|    | Layer      |   Nodes |   Links |   ⟨k⟩ |     σ |
|---:|:-----------|--------:|--------:|------:|------:|
|  0 | calls      |     536 |     621 |  2.32 |  1.83 |
|  1 | fb_friends |     800 |    6418 | 16.05 | 13.39 |
|  2 | sms        |     568 |     697 |  2.45 |  1.76 |
|    | Layer    |   Nodes |   Links |   ⟨k⟩ |    σ |
|---:|:---------|--------:|--------:|------:|-----:|
|  0 | lunch    |      60 |     193 |  6.43 | 2.99 |
|  1 | facebook |      32 |     124 |  7.75 | 3.75 |
|  2 | work     |      60 |     194 |  6.47 | 5.32 |
|    | Layer      |   Nodes |   Links |   ⟨k⟩ |    σ |
|---:|:-----------|--------:|--------:|------:|-----:|
|  0 | advice     |     215 |     449 |  4.18 | 2.94 |
|  1 | discussion |     231 |     498 |  4.31 | 2.55 |
|  2 | friend     |     228 |     423 |  3.71 | 1.81 |
|    | Layer     |   Nodes |   Links |   ⟨k⟩ |     σ |
|---:|:----------|--------:|--------:|------:|------:|
|  0 | Lufthansa |     106 |     244 |  4.60 | 11.19 |
|  1 | Ryanair 

## 2. Benchmark Indices

In [3]:
from maapy.indices import *
import numpy as np
import pandas as pd
import os
from itertools import product
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import networkx as nx


eta_grid = get_positive_eta_grid()
simple_results = get_simple_results()
weighted_etas = get_weighted_etas()

adj = lambda g: nx.adjacency_matrix(g).toarray()

scores = [adamic_adar, common_neighbors, has_common_neighbors, jaccard_coefficient, preferential_attachment]

for dataset, layers in datasets[5:]:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    for layerID, layerName in ds.layer_dict.items():
        print(f"Processing {dataset} - {layerName}")

        # Construct the true vector
        true = pd.Series(False, index=[(u, v) for u, v in ds.ncombs_.T])
        true[list(ds.mg[layerID].edges)] = True

        ### Single Layer Scores ###
        a = adj(ds.mg[layerID])
        d = np.array(list(ds.mg[layerID].degree))[:, -1]

        for s in scores:
            simple_results.loc[(dataset, layerID, s.__name__, "One-Layer", "roc_auc")] = roc_auc_score(
                true, s(a, d, ds.ncombs_)
            )

        ### Simple and Weighted Aggregate Scores ###
        A = ds.A()
        D = [np.array(list(g.degree))[:, -1] for g in ds.mg.values()]

        for s in scores:
            score_values = np.array([s(a, d, ds.ncombs_) for a, d in zip(A, D)])

            simple_results.loc[(dataset, layerID, s.__name__, "Simple-Average", "roc_auc")] = roc_auc_score(
                true, score_values.sum(axis=0)
            )

            ### Weighted Aggregate Scores ###
            for i, eta in tqdm(enumerate(eta_grid), desc=f"Eta Grid for {s.__name__}", total=len(eta_grid)):
                idx = (dataset, layerID, s.__name__, "roc_auc", i)
                try:
                    if np.isnan(weighted_etas.loc[idx]):
                        weighted_etas.loc[idx] = roc_auc_score(true, np.average(score_values, axis=0, weights=eta))
                except KeyError:
                    weighted_etas.loc[idx] = roc_auc_score(true, np.average(score_values, axis=0, weights=eta))

Processing lon - tube


Eta Grid for preferential_attachment: 100%|██████████| 5151/5151 [00:00<00:00, 92333.20it/s]


Processing lon - overground


Eta Grid for preferential_attachment: 100%|██████████| 5151/5151 [00:00<00:00, 87062.66it/s]


Processing lon - light_railway


...

Eta Grid for preferential_attachment: 100%|██████████| 5151/5151 [00:00<00:00, 92146.92it/s]


Processing vic - best_friends


Eta Grid for preferential_attachment: 100%|██████████| 5151/5151 [00:00<00:00, 92545.59it/s]


Processing vic - work_with


Eta Grid for preferential_attachment: 100%|██████████| 5151/5151 [00:00<00:00, 63043.07it/s]


## 3. Vanilla and Multiplex Adamic-Adar Index

The Multiplex Adamic-Adar Index (MAA) is a generalization of the Adamic-Adar Index for multiplex networks. The Adamic-Adar Index for edges $(u, v)$ in layer $\gamma$ is defined as:

$$
\operatorname{MAA}(u, v, \gamma ;\eta)=\sum_{\alpha, \beta} \sum_{w \in \mathcal{T}_{\alpha \beta}} \frac{\eta_{\gamma \alpha} \eta_{\gamma \beta}}{\sqrt{\langle k\rangle_\alpha\langle k\rangle_\beta}} \frac{1}{\sqrt{\ln \left(k_w^{(\alpha)} \right) \ln \left(k_w^{(\beta)}\right)}}
$$

- where $\eta_{\gamma \alpha}$ and $\eta_{\gamma \beta}$ are the coefficients that control the influence of the layers $\alpha$ and $\beta$ on layer $\gamma$,
- $\langle k\rangle_\alpha$ and $\langle k\rangle_\beta$ are the average degrees of the nodes in layers $\alpha$ and $\beta$, respectively,
- $k_w^{(\alpha)}$ and $k_w^{(\beta)}$ are the degrees of the node $w$ in layers $\alpha$ and $\beta$, respectively.

In [4]:
for dataset, layers in datasets:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    for layerID, layerName in ds.layer_dict.items():

        # Construct the true vector
        true = pd.Series(False, index=[(u, v) for u, v in ds.ncombs_.T])
        true[list(ds.mg[layerID].edges)] = True

        maa = multiplex_adamic_adar(ds, layers=[layerID])[0]
        maa = np.asarray(maa[ds.ncombs_[0], ds.ncombs_[1]])[0]

        idx = (dataset, layerID, "multiplex_adamic_adar", "Simple-Average", "roc_auc")
        simple_results.loc[idx] = roc_auc_score(true, maa)

### 3.1. Finding the best hyperparameters

In [5]:
for dataset, layers in datasets:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    eta = np.ones((ds.L, ds.L)) / ds.L
    for layerID, layerName in ds.layer_dict.items():

        # Construct the true vector
        true = pd.Series(False, index=[(u, v) for u, v in ds.ncombs_.T])
        true[list(ds.mg[layerID].edges)] = True

        for etaID, eta_row in tqdm(enumerate(eta_grid), desc=f"Dataset {dataset}-{layerName}", total=len(eta_grid)):
            idx = (dataset, layerID, "multiplex_adamic_adar", "roc_auc", etaID)
            if np.isnan(weighted_etas.loc[idx]):

                eta[layerID] = eta_row / 100
                maa = multiplex_adamic_adar(ds, eta=eta, layers=[layerID])[0]
                maa = np.asarray(maa[ds.ncombs_[0], ds.ncombs_[1]])[0]
                weighted_etas.loc[idx] = roc_auc_score(true, maa)

        weighted_etas.to_pickle("results/weighted_etas.pkl")

Dataset vic-work_with: 100%|██████████| 5151/5151 [00:00<00:00, 90863.77it/s]


## 4. Results

### 4.1. Simple Average

In [6]:
table1 = simple_results.reset_index()
table1.columns = ["dataset", "layerID", "score", "aggregation", "metric", "value"]
table1["layerName"] = table1.apply(lambda x: dict(datasets)[x["dataset"]][x["layerID"]], axis=1)
table1.sort_values(table1.columns.tolist(), inplace=True)

# Simple Average
table1[table1["aggregation"].isin(["Simple-Average"])].pivot_table(
    columns=["dataset", "layerName"], index="score", values="value"
).style.background_gradient("coolwarm", axis=0).format("{:.3f}")

### 4.2. Weighted Average

In [7]:
table2 = weighted_etas.reset_index().groupby(["dataset", "layerID", "score", "metric"]).max()
table2 = table2.reset_index()
table2.columns = ["dataset", "layerID", "score", "metric", "etaID", "value"]
table2["layerName"] = table2.apply(lambda x: dict(datasets)[x["dataset"]][x["layerID"]], axis=1)
table2.sort_values(table2.columns.tolist(), inplace=True)

# Simple Average
table2.pivot_table(columns=["dataset", "layerName"], index="score", values="value").style.background_gradient(
    "coolwarm", axis=0
).format("{:.3f}")

## Allowing for negative hyperparameters

In [8]:
bothside_etas = get_bothside_etas()
bothsides_eta_grid = get_bothsides_eta_grid()

for dataset, layers in datasets:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    for layerID, layerName in ds.layer_dict.items():
        print(f"Processing {dataset} - {layerName}")

        # Construct the true vector
        true = pd.Series(False, index=[(u, v) for u, v in ds.ncombs_.T])
        true[list(ds.mg[layerID].edges)] = True

        ### Simple and Weighted Aggregate Scores ###
        A = ds.A()
        D = [np.array(list(g.degree))[:, -1] for g in ds.mg.values()]

        for s in scores:
            score_values = np.array([s(a, d, ds.ncombs_) for a, d in zip(A, D)])

            ### Weighted Aggregate Scores ###
            for i, eta in tqdm(
                enumerate(bothsides_eta_grid), desc=f"Eta Grid for {s.__name__}", total=len(bothsides_eta_grid)
            ):
                idx = (dataset, layerID, s.__name__, "roc_auc", i)
                try:
                    if np.isnan(bothside_etas.loc[idx]):
                        pred = np.average(score_values, axis=0, weights=eta)
                        bothside_etas.loc[idx] = roc_auc_score(true, pred)
                except KeyError:
                    pred = np.average(score_values, axis=0, weights=eta)
                    bothside_etas.loc[idx] = roc_auc_score(true, pred)

            # Save results
            bothside_etas.to_pickle("results/bothside_etas.pkl")

Processing cns - calls


Eta Grid for preferential_attachment: 100%|██████████| 1106/1106 [00:00<00:00, 92995.61it/s]


Processing cns - fb_friends


Eta Grid for preferential_attachment: 100%|██████████| 1106/1106 [00:00<00:00, 91921.30it/s]


Processing cns - sms


...

Eta Grid for preferential_attachment: 100%|██████████| 1106/1106 [00:00<00:00, 88650.44it/s]


Processing vic - best_friends


Eta Grid for preferential_attachment: 100%|██████████| 1106/1106 [00:00<00:00, 85876.93it/s]


Processing vic - work_with


Eta Grid for preferential_attachment: 100%|██████████| 1106/1106 [00:00<00:00, 88183.64it/s]


In [9]:
for dataset, layers in datasets:
    ds = MultiplexGraphDataset(dataset, layers, force_undirected=True)
    eta = np.ones((ds.L, ds.L)) / ds.L
    for layerID, layerName in ds.layer_dict.items():

        # Construct the true vector
        true = pd.Series(False, index=[(u, v) for u, v in ds.ncombs_.T])
        true[list(ds.mg[layerID].edges)] = True

        for etaID, eta_row in tqdm(
            enumerate(bothsides_eta_grid), desc=f"{dataset}-{layerName}", total=len(bothsides_eta_grid)
        ):
            idx = (dataset, layerID, "multiplex_adamic_adar", "roc_auc", etaID)
            # if np.isnan(best_eta2.loc[idx]):
            try:
                if np.isnan(bothside_etas.loc[idx]):
                    eta[layerID] = eta_row
                    maa = multiplex_adamic_adar(ds, eta=eta, layers=[layerID])[0]
                    maa = np.asarray(maa[ds.ncombs_[0], ds.ncombs_[1]])[0]
                    bothside_etas.loc[idx] = roc_auc_score(true, maa)
            except KeyError:
                eta[layerID] = eta_row
                maa = multiplex_adamic_adar(ds, eta=eta, layers=[layerID])[0]
                maa = np.asarray(maa[ds.ncombs_[0], ds.ncombs_[1]])[0]
                bothside_etas.loc[idx] = roc_auc_score(true, maa)

        # Save results
        bothside_etas.to_pickle("results/bothside_etas.pkl")

vic-work_with: 100%|██████████| 1106/1106 [00:00<00:00, 83783.06it/s]


In [10]:
table3 = bothside_etas.reset_index().groupby(["dataset", "layerID", "score", "metric"]).max()
table3 = table3.reset_index()
table3.columns = ["dataset", "layerID", "score", "metric", "etaID", "value"]
table3["layerName"] = table3.apply(lambda x: dict(datasets)[x["dataset"]][x["layerID"]], axis=1)
table3.sort_values(table3.columns.tolist(), inplace=True)

# Simple Average
table3.pivot_table(columns=["dataset", "layerName"], index="score", values="value").style.background_gradient(
    "coolwarm", axis=0
).format("{:.3f}")